In [1]:
import requests
from datetime import datetime
import hashlib
import time

In [2]:
base_url = 'https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByPin'
telegram_api_url = 'https://api.telegram.org/bot1794047380:AAGrpZN8ITFj_wU5DYgqpyhSkmmkHtRLlig/sendMessage?chat_id=-1001311953767&text='
pin_codes = ['400068', '400092', '400103']


In [3]:
def cowin_url_response(pincode):
    url = base_url+'?pincode='+pincode+'&date='+datetime.now().strftime('%d-%m-%Y')
    response = requests.get(url)
    response_text = response.text.encode('utf 8')   #For hashing
    response_json = response.json()   #For getting individual center deets
#     for center in response_json["centers"]:
# #         print('\n',center['center_id'], center['name'], ' ')
#         center_name = center['name']
#         for session in center['sessions']:
# #             print(session['available_capacity_dose1'], session['min_age_limit'])
#             min_age_limit = session['min_age_limit']
#             available = session['available_capacity_dose1'] 
    hashed_response = hashlib.sha256(response_text).hexdigest()
            
    return (hashed_response, response_json)

def send_telegram_message(message):
    telegram_url = telegram_api_url + message
    try: 
        response = requests.get(telegram_url)
    except:
        print('Telegram issue')
    print(response.text)
    
def center_loop(response_json):
#     session_hash
    for center in response_json['centers']:
        center_name = center['name']
        pin = center['pincode']
        cat = center['fee_type']
        for session in center['sessions']:
            min_age_limit = session['min_age_limit']
            available_1 = session['available_capacity_dose1'] 
            available_2 = session['available_capacity_dose2'] 
            vaccine_type = session['vaccine']
            vax_date = session['date']
            vax_date = datetime.strptime(vax_date,'%d-%m-%Y').strftime('%b %d, %Y')
            if available_1 > 0 or available_2 > 0:
                if min_age_limit == 18:
                    print(f'{available_1+available_2} {cat} {vaccine_type} vaccine(s) ({available_1} for FIRST DOSE | {available_2} for SECOND DOSE) for {vax_date} currently available for 18+ at {center_name} in zip code {pin}')
                    send_telegram_message(f'{available_1+available_2} {cat} {vaccine_type} vaccine(s) ({available_1} for FIRST DOSE | {available_2} for SECOND DOSE) for {vax_date} currently available for 18-plus at {center_name} in zip code {pin}')
    #                 break
                elif min_age_limit == 30:
                    print(f'{available_1+available_2} {cat} {vaccine_type} vaccine(s) ({available_1} for FIRST DOSE | {available_2} for SECOND DOSE) for {vax_date} currently available for 30+ at {center_name} in zip code {pin}')
                    send_telegram_message(f'{available_1+available_2} {cat} {vaccine_type} vaccine(s) ({available_1} for FIRST DOSE | {available_2} for SECOND DOSE) for {vax_date} currently available for 30-plus at {center_name} in zip code {pin}')
    #                 break
                elif min_age_limit == 45:
                    print(f'{available_1+available_2} {cat} {vaccine_type} vaccine(s) ({available_1} for FIRST DOSE | {available_2} for SECOND DOSE) for {vax_date} currently available for 45+ at {center_name} in zip code {pin}')
                    send_telegram_message(f'{available_1+available_2} {cat} {vaccine_type} vaccine(s) ({available_1} for FIRST DOSE | {available_2} for SECOND DOSE) for {vax_date} currently available for 45-plus at {center_name} in zip code {pin}')
    #                 break
                else:
                    continue

In [4]:
old_state = []    #list of previous(initial) state for all 3 pincodes
new_state = []    #list of new state for all 3 pincodes

for pin in pin_codes:
    old_state.append(cowin_url_response(pin))
    
for state in old_state:
    center_loop(state[1])

# old_state = cowin_url_response('400092')
# center_loop(old_state[1])


while True:
    time.sleep(10)
#     (t_i=3*n_r | t_i: minimum time interval b/w requests | n_r: no. of requests after every t_i)
#     since limit is 100 requests every 5mins(300secs)
#     new_state = cowin_url_response('400092')
    new_state = []    #doesn't this erase old_state list as well? -> nope
    for pin in pin_codes:
        new_state.append(cowin_url_response(pin))
    
    for i in range(len(pin_codes)):
        if old_state[i][0] != new_state[i][0]:
            center_loop(new_state[i][1])
#  ###########
#     if old_state[0] != new_state[0]:
#         center_loop(new_state[1])              
####
#     else:
#         print(new_state[0])
    
    old_state = new_state
    
    continue                
    

KeyboardInterrupt: 

In [ ]:
# old_state_1 = cowin_url_response('400068')
# old_state_2 = cowin_url_response('400092')
# old_state_3 = cowin_url_response('400103')

# while True:   
#     time.sleep(10)
# 
# #     since limit is 100 requests every 5mins(300secs)
#     new_state_1 = cowin_url_response('400068')
#     new_state_2 = cowin_url_response('400092')
#     new_state_3 = cowin_url_response('400103')
    
#     if new_state_1[0] != old_state_1[0]:
#         if int(new_state_1[2])==18 and int(new_state_1[3])>0:
#             print('Vaccine available in 400068')
#             send_telegram_message('Vaccine available in 400068')
#     elif new_state_2[0] != old_state_2[0]:
#         if int(new_state_2[2])==18 and int(new_state_2[3])>0:
#             print('Vaccine available in 400092')
#             send_telegram_message('Vaccine available in 400092')
#     elif new_state_3[0] != old_state_3[0]:
#         if int(new_state_3[2])==18 and int(new_state_3[3])>0:
#             print('Vaccine available in 400103')
#             send_telegram_message('Vaccine available in 400103')  
#     else:
#         print(new_state_1, '\n', new_state_2, '\n', new_state_3)
    
#     old_state_1 = new_state_1
#     old_state_2 = new_state_2
#     old_state_3 = new_state_3    
    
#     continue
    
    
    

In [ ]:
# https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByPin?pincode=400001&date=21-06-2021